## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection command:
indy0: False
panda1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.82746387, -0.02422198, -0.5609963 ,  0.64826906],
        [ 0.17269257,  0.9616252 ,  0.21319999, -0.16085054],
        [ 0.534304  , -0.27329516,  0.7998931 ,  0.08972335],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.05)

   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker


## add indy tool

In [8]:
from pkg.utils.code_scraps import add_indy_sweep_tool
add_indy_sweep_tool(gscene, "indy0", face_name="brush_face")

## add sweep face

In [9]:
track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [10]:
gscene.update_markers_all()

## add wp

In [11]:
TOOL_DIM = [0.06, 0.10]
TACK_WIDTH = 0.17
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
WP_REF_M = np.array([0, WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TRACK_DIM[0] - TOOL_DIM[0]/2,TACK_WIDTH,TRC_THIC), 
                             center=tuple(WP_REF_M+[0,TRACK_STEP*i_trc])+(1e-2,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))

In [12]:
gscene.update_markers_all()

## add blocking structure on table

In [13]:
# gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
#                             center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
#                             collision=True, fixed=True)

## add box

In [14]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [15]:
# # give clearance from the track surface
# s_builder.give_clearance(track, gtem_dict.values())

In [16]:
gscene.update_markers_all()

## Register binders

In [17]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [18]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,0.00), 
                     rpy=(0,np.pi,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)

## add objects

In [19]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [20]:
for gname in sorted(gtem_dict.keys()):
    pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [21]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [22]:
sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)

### planners

In [23]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [24]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [rcheck, gcheck, tcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [25]:
mplan.motion_filters = checkers_all

In [26]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [27]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

from pkg.utils.code_scraps import use_current_place_point_only, use_current_sub_binders_only
# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

('track_face', 'track_face', 0, 0, 0)


In [28]:
pscene.subject_name_list

['box1', 'box2', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [29]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [30]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan & show

In [31]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
mplan.reset_log(False)
gtimer.reset()

In [32]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=20, multiprocess=False, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        if len(snode_schedule_all)==0:
            gtimer.toc("firstmove")
    snode_schedule_all.append(snode_schedule)
#     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
#     t_exe.start()
# t_exe.join()

('track_face', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try transition motion
transition motion tried: True
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = success
branching: 0->1 (0.13/20.0 s, steps/e

result: 0 - ('track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = success
branching: 0->3 (0.35/20.0 s, steps/err: 56(72.8461742401 ms)/0.00126887739475)
try: 3 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try joint motion
joint motion tried: True
result: 3 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = success
branching: 3->4 (0.44/20.0 s, steps/err: 57(91.0799503326 ms)/5.92493037107e-16)
try: 3 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 1, 0)
result: 3 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 1, 0) = fail
try: 4 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 1, 0)
result: 4 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 1, 0) = fail
try: 1 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
result: 1 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face'

joint motion tried: True
result: 5 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
branching: 5->6 (0.49/20.0 s, steps/err: 72(124.201059341 ms)/4.5627802337e-16)
try: 5 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
result: 5 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
try: 6 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
result: 6 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
try: 3 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
result: 3 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
try: 5 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
result: 5 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
try: 3 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
try tra

In [33]:
print(gtimer)

detect: 	180.0 ms/1 = 179.868 ms (179.868/179.868)
firstmove: 	1121.0 ms/1 = 1120.669 ms (1120.669/1120.669)
plan0: 	947.0 ms/1 = 947.218 ms (947.218/947.218)
initialize_memory: 	0.0 ms/3 = 0.044 ms (0.03/0.056)
init_search: 	6.0 ms/3 = 1.876 ms (1.558/2.482)
test_connection: 	2620.0 ms/74 = 35.405 ms (0.89/345.768)
plan1: 	1321.0 ms/1 = 1320.829 ms (1320.829/1320.829)
plan2: 	1157.0 ms/1 = 1157.252 ms (1157.252/1157.252)



## mix full schedule

In [34]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

with gtimer.block("mix_schedule_safe"):
    safe_mixed = mix_schedule(mplan, snode_schedule_cat)

print(gtimer)

ppline.play_schedule(safe_mixed, period=0.001)

mix_schedule_safe: 	256.0 ms/1 = 256.315 ms (256.315/256.315)



### Run fully mixed schedule

In [35]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)
time.sleep(1)
crob.grasp(True, True)
time.sleep(1)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose)
crob.joint_move_make_sure(crob.home_pose)

connection command:
indy0: True
panda1: True
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [36]:
ppline.execute_schedule(safe_mixed)

binder: grip1
rname: panda1
binder: track_face
rname: None
binder: None
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: track_face
rname: None
binder: None
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: None
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: grip1
rname: panda1
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: grip1
rname: panda1
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.

## Plan & execute

In [37]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)
time.sleep(1)
crob.grasp(True, True)
time.sleep(1)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose)

connection command:
indy0: True
panda1: True
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [38]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=20, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        gtimer.toc("firstmove")
    t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,))
    t_exe.start()
    snode_schedule_all.append(snode_schedule)
t_exe.join()

('track_face', 'track_face', 0, 0, 0)
Use 20/20 agents
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 

try transition motion
result: 1 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
result: 1 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
try: 1 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 1 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try transition motion
transition motion tried: True
try transition motion
result: 1 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
branching: 1->3 (0.83/20.0 s, steps/err: 54(112.287998199 ms)/0.000939979234558)
try: 3 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try joint motion
transition motion tried: True
result: 1 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
branching: 1->4 (0.87/20.0 s, steps/err: 46(91.292142868 ms)/0.00119771288939)
try: 3 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1,

result: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
result: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
try: 11 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
branching: 9->11 (1.26/20.0 s, steps/err: 65(121.381044388 ms)/1.14647669424e-15)
joint motion tried: True
try: 4 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 11 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
try: 5 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 4 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 6 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
try: 0 - ('track_face', '

try transition motion
try: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
branching: 5->20 (1.69/20.0 s, steps/err: 63(373.010873795 ms)/0.00172730165468)
result: 20 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
try: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try transition motion
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 5 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 4 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 6 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 5 - ('track_face', 'track_face'

try: 14 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 5 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
try transition motion
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try joint motion
constrained motion tried: True
result: 17 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
branching: 5->28 (2.16/20.0 s, steps/err: 64(644.140958786 ms)/0.00144467319055)
result: 28 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
result: 26 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
result: 14 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try transition motion
try: 20 - ('track_face', 

...

try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
result: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
result: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
======================= terminated 5: first answer acquired ===============================
result: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try transition motion
result: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
result: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try: 0 - ('grip1', 'track_

constrained motion tried: True
result: 3 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = success
result: 1 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = success
branching: 1->9 (0.94/20.0 s, steps/err: 51(267.124176025 ms)/0.00173416008092)
constrained motion tried: True
branching: 3->10 (0.94/20.0 s, steps/err: 46(194.588899612 ms)/0.00180670732)
try: 10 - ('track_face', 'track_face', 2, 2, 1)->('track_face', 'track_face', 2, 2, 2)
result: 8 - ('track_face', 'track_face', 2, 2, 1)->('track_face', 'track_face', 2, 2, 2) = success
result: 7 - ('grip1', 'track_face', 2, 2, 1)->('grip1', 'track_face', 2, 2, 2) = success
try: 9 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
branching: 7->11 (0.97/20.0 s, steps/err: 28(107.440948486 ms)/0.00103331477666)
branching: 8->12 (0.96/20.0 s, steps/err: 28(42.8669452667 ms)/0.00217386774641)
try joint motion
try constrained motion
++ adding return motion to acquire

======================= terminated 15: first answer acquired from other agent ===============================
======================= terminated 13: first answer acquired from other agent ===============================
transition motion tried: True
======================= terminated 16: first answer acquired from other agent ===============================
transition motion tried: True
joint motion tried: True
======================= terminated 17: first answer acquired from other agent ===============================
result: 16 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = success
result: 17 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = success
result: 15 - ('track_face', 'track_face', 2, 2, 2)->('grip1', 'track_face', 2, 2, 2) = success
branching: 16->22 (1.4/20.0 s, steps/err: 58(232.871055603 ms)/0.000966274313282)
branching: 17->20 (1.41/20.0 s, steps/err: 29(194.365978241 ms)/5.8433486347e-16)
branching: 15->21 (1.41/20.0 s,

binder: track_face
rname: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


Process Process-50:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 182, in __search_loop
    self.add_return_motion(snode_new)
  File "pkg/planning/pipeline.py", line 302, in add_return_motion
    display=False, timeout=timeout)
  File "pkg/planning/pipeline.py", line 204, in test_connection
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/motion/interface.py", line 102, in plan_transition
    redundancy_values=redundancy_values, **kwargs)
  File "pkg/planning/motion/moveit/moveit_planner.py", line 197, in plan_algorithm
    group_name, tuple(to_Q), tuple(from_Q), timeout=timeout_joint, **kwargs)
  File "pkg/planning/motion/moveit/moveit_py.py", line 127, in plan_joint_motion_py
  

In [39]:
print(gtimer)

detect: 	198.0 ms/1 = 198.243 ms (198.243/198.243)
firstmove: 	3569.0 ms/1 = 3568.683 ms (3568.683/3568.683)
plan0: 	2651.0 ms/1 = 2650.955 ms (2650.955/2650.955)
initialize_memory: 	79.0 ms/3 = 26.223 ms (6.581/64.342)
init_search: 	32.0 ms/3 = 10.651 ms (10.286/10.963)
start_process: 	1738.0 ms/3 = 579.171 ms (513.057/674.066)
plan1: 	9146.0 ms/1 = 9146.424 ms (9146.424/9146.424)
plan2: 	1338.0 ms/1 = 1338.221 ms (1338.221/1338.221)

